In [1]:
import landsat_prep as lp
import geograph as gg
import numpy as np
import random
import torch

/home/hbaier/anaconda3/envs/caoe/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# Set variables
ISO = "MEX"
ADM_ID = "MEX-ADM2-1590546715-B7"
IC = "LANDSAT/LT05/C01/T1"
YEAR = "2010"
MONTH = "1"
GB_PATH = "./data/MEX/ipumns_shp.shp"

In [28]:
# gdf = gpd.read_file("./data/MEX/geo2_mx1960_2015.shp")
# gdf = gdf[["GEOLEVEL2", "geometry"]]
# gdf.columns = ["shapeID", "geometry"]
# gdf.to_file("./ipumns_shp.shp")

# lp.downloadGB(iso = "MEX", 
#               adm = "2")

In [2]:
# ADM_ID = "484001001"
# lp.prep_landsat(GB_PATH, ISO, ADM_ID, "2010", "1", IC, v = False)

# ADM_ID = "484001003"
# lp.prep_landsat(GB_PATH, ISO, ADM_ID, "2010", "1", IC, v = False)

# ADM_ID = "484014116"
# lp.prep_landsat(GB_PATH, ISO, ADM_ID, "2010", "1", IC, v = False)

# ADM_ID = "484014046"
# lp.prep_landsat(GB_PATH, ISO, ADM_ID, "2010", "1", IC, v = False)

In [25]:
class GraphLoader():

    class Dataset():
        def __init__(self, x, y, edge_index, batch):
            self.x = x
            self.y = y
            self.edge_index = edge_index
            self.batch = batch

    def __init__(self, data_dir, iso, dta_path, batch_size):

        self.iso = iso
        self.data_dir = os.path.join(data_dir, iso)
        self.batch_size = batch_size
        self.munis = [i for i in os.listdir(self.data_dir) if os.path.isdir(os.path.join(self.data_dir, i))]
        self.data_dir = os.path.join(data_dir, iso)
        self.image_graphs = [gg.ImageGraph(self.data_dir, i, dta = dta_path) for i in self.munis] 
        self.indexes = [i for i in range(0, len(self.munis))]
        random.shuffle(self.indexes)
        self.indexes = [self.indexes[i:i + self.batch_size] for i in range(0, len(self.indexes), self.batch_size)]

        self.data = []
        [self.data.append(self.__load_graph(np.array(self.image_graphs)[batch])) for batch in self.indexes]


    def __load_graph(self, batch):

        # X's 
        xs = torch.cat([i.x for i in batch])

        # Y's
        ys = torch.tensor([i.y for i in batch]).view(-1, 1)

        # Batch ID's
        batch_ids = []
        node_nums = [i.num_nodes for i in batch]
        for i in range(len(node_nums)):
            batch_ids.append(np.array([i for n in range(node_nums[i])], dtype = np.float32))        
        batch_ids = torch.tensor(np.concatenate(batch_ids))

        # Edge Indices
        all_edge_indices = []
        edge_indices = [i.edge_list for i in batch]
        for i in range(len(edge_indices)):
            all_edge_indices.append(np.array(edge_indices[i]) + np.sum(np.array(node_nums)[:i]))
        edge_indices = torch.tensor(np.concatenate(all_edge_indices))
        
        return self.Dataset(xs, ys, edge_indices, batch_ids)

In [26]:
g = GraphLoader("./data/", "MEX", "/home/hbaier/Desktop/geograph/test/migration_data.json", 4).data
g

In [27]:
g[0].y

tensor([[ 3721.],
        [42055.],
        [ 3417.],
        [11992.]])